In [39]:
from livescore_scraper import scrape
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
df = scrape('2020-08-21', '2021-04-23')
df

,date,home_team,home_score,away_team,away_score,played
0,2020-08-21,Bordeaux,0,Nantes,0,True
1,2020-08-22,Dijon,0,Angers,1,True
2,2020-08-22,Lille,1,Rennes,1,True
3,2020-08-23,Monaco,2,Reims,2,True
4,2020-08-23,Lorient,3,Strasbourg,1,True
...,...,...,...,...,...,...
326,2021-04-18,Nimes,1,Strasbourg,1,True
327,2021-04-18,Reims,0,Metz,0,True
328,2021-04-18,Bordeaux,0,Monaco,3,True
329,2021-04-18,Nantes,1,Lyon,2,True


In [58]:
def analyze_matches(team):
    
    fig = plt.figure(figsize=(15,15))
    gs = fig.add_gridspec(4, 3)
    gs.update(hspace = 0.35, wspace = 0.4)
    
    dft = df[((df.home_team == team) | (df.away_team == team)) & (df.played == True)]
    
    df_wins_home = dft[(dft.home_team == team) & (dft.home_score > dft.away_score)]
    df_wins_away = dft[(dft.away_team == team) & (dft.home_score < dft.away_score)]
    
    df_even_home = dft[(dft.home_team == team) & (dft.home_score == dft.away_score)]
    df_even_away = dft[(dft.away_team == team) & (dft.home_score == dft.away_score)]
    
    df_lose_home = dft[(dft.home_team == team) & (dft.home_score < dft.away_score)]
    df_lose_away = dft[(dft.away_team == team) & (dft.home_score > dft.away_score)]
    
    nb_wins = len(df_wins_home) + len(df_wins_away)
    nb_even = len(df_even_home) + len(df_even_away)
    nb_lose = len(df_lose_home) + len(df_lose_away)
    
    wl_ratio = np.round(nb_wins/nb_lose)*100
    
    nb_goals = dft[(dft.home_team == team)].home_score.sum() + dft[(dft.away_team == team)].away_score.sum()
    nb_taken = dft[(dft.home_team == team)].away_score.sum() + dft[(dft.away_team == team)].home_score.sum()
    
    goals_ratio = np.round(nb_goals/nb_taken)*100
    goal_diff = nb_goals - nb_taken
    
    wl_team = []
    for t in set(dft.home_team.unique()).union(set(dft.away_team.unique())):
        home_wins = dft[(dft.home_team == team) & (dft.away_team == t) & (dft.home_score > dft.away_score)]
        away_wins = dft[(dft.home_team == t) & (dft.away_team == team) & (dft.home_score < dft.away_score)]
        
        home_lose = dft[(dft.home_team == team) & (dft.away_team == t) & (dft.home_score < dft.away_score)]
        away_lose = dft[(dft.home_team == t) & (dft.away_team == team) & (dft.home_score > dft.away_score)]
        
        home_even = dft[(dft.home_team == team) & (dft.away_team == t) & (dft.home_score == dft.away_score)]
        away_even = dft[(dft.home_team == t) & (dft.away_team == team) & (dft.home_score == dft.away_score)]
        
        wl_team.append((
            t,
            len(home_wins), len(away_wins), len(home_wins)+len(away_wins),
            len(home_even), len(away_even), len(home_even)+len(away_even),
            len(home_lose), len(away_lose), len(home_lose)+len(away_lose)
        ))
        
    df_wl_team = pd.DataFrame(wl_team, columns=['opponent',
                                                'home_wins', 'away_wins', 'total_wins',
                                               'home_even', 'away_even', 'total_even',
                                               'home_lose', 'away_lose', 'total_lose'])
        
    ax0 = fig.add_subplot(gs[1, 0])
    sns.barplot(x='opponent', y='total_wins', data=df_wl_team, ax=ax0, color='royalblue')
    plt.xticks(rotation=-15, ha='left')
    
    ax1 = fig.add_subplot(gs[1, 1])
    sns.barplot(x='opponent', y='total_even', data=df_wl_team, ax=ax1, color='royalblue')
    plt.xticks(rotation=-15, ha='left')
    
    ax2 = fig.add_subplot(gs[1, 2])
    sns.barplot(x='opponent', y='total_lose', data=df_wl_team, ax=ax2, color='royalblue')
    plt.xticks(rotation=-15, ha='left')

In [ ]:
analyze_matches('Paris Saint-Germain')